In [238]:
from lxml import etree
from operator import itemgetter
import math,re,codecs
debug=False

In [239]:
parser = etree.XMLParser(remove_blank_text=True)

In [240]:
nomLoc="/Users/gilles/Downloads/21739864.gpx"
wptChildren=["name","urlname"]

In [241]:
def localName(element):
    return etree.QName(element).localname

In [242]:
def Dec2Min(coord):
    degreSign=""
    decDegres,intDegres=math.modf(coord)
    minutes=abs(decDegres)*60
    return "%d%s %5.3f"%(intDegres,degreSign,minutes)

In [243]:
def CoordDec2Min(lat,lon):
    sep=" "
    coordLat=Dec2Min(lat)
    coordLon=Dec2Min(lon)
    if coordLat.startswith("-"):
        coordLat="S"+coordLat[1:]
    else:
        coordLat="N"+coordLat
    if coordLon.startswith("-"):
        coordLon="W"+coordLon[1:]
    else:
        coordLon="E"+coordLon
    return "%s%s%s"%(coordLat,sep,coordLon)

In [244]:
def dictLatLon2Coord(dictLatLon):
    return CoordDec2Min(float(dictLatLon["lat"]),float(dictLatLon["lon"]))

In [245]:
Dec2Min(-1.57305)

'-1 34.383'

In [246]:
tree = etree.parse(nomLoc)
root = tree.getroot()
nsRoot=root.nsmap[None]
nsGC="http://www.groundspeak.com/cache/1/0/1"
print root.tag

{http://www.topografix.com/GPX/1/0}gpx


In [247]:
textPrint=False
textCount=True
dictLogs={}
logsTexte=[]
texteWords={}
noNum=0
noNumIndex="abcdefghijklmnopqrstuvwxyz"
for wpt in root.findall("{%s}wpt"%nsRoot):
    cacheName=wpt.find("{%s}desc"%nsRoot).text
    cacheCode=u"http://coord.info/%s"%wpt.find("{%s}name"%nsRoot).text
    cache=wpt.find("{%s}cache"%nsGC)
    logs=cache.find("{%s}logs"%nsGC)
    logList=logs.findall("{%s}log"%nsGC)
    for log in logList: 
        logType=log.find("{%s}type"%nsGC)
        if logType.text in ["Found it","Attended"]:
            logId=log.attrib["id"]
            logHref=cacheCode+"#%s"%logId
            logText=log.find("{%s}text"%nsGC)
            texteLog=logText.text
            logStamp=log.find("{%s}date"%nsGC).text
            logDate=log.find("{%s}date"%nsGC).text.split("T")[0]
            (year,month,day)=logDate.split("-")[::-1]
            dateLog=u"%s/%s/%s"%(day,month,year)
#            print logStamp
            if logStamp in dictLogs:
#                print "duplicate stamp, add cache to list", logStamp
                dictLogs[logStamp].append([cacheName,texteLog,dateLog,cacheCode,logHref])
            else:
                dictLogs[logStamp]=[[cacheName,texteLog,dateLog,cacheCode,logHref]]

numCache=1
listeLogs=[]
for stamp in sorted(dictLogs.keys()):
    for log in sorted(dictLogs[stamp],key=itemgetter(4)):
        listeLogs.append([numCache]+log)
        numCache+=1
        logsTexte.append(log[1])
        if textCount:
            for element in re.split(ur"['\s,.-]+",texteLog):
                word=element.lower()
                if not word in texteWords: texteWords[word]=0
                texteWords[word]+=1

if textPrint:
    print len(logsTexte)
    with codecs.open("test/logsTexte.txt","w",encoding='utf8') as output:
        output.writelines(logsTexte[:5000])
        output.writelines(logsTexte[5000:10000])
        output.writelines(logsTexte[10000:15000])
        output.writelines(logsTexte[15000:])

In [248]:
logsTexte

[u'\U0001f600  Ma premi\xe8re cache trouv\xe9e ....merci',
 u'Tr\xe8s beau lavoir, bel endroit pour une g\xe9ocache. Merci',
 u'J ai visit\xe9 le ch\xe2teau 2 jours avant de d\xe9couvrir le geocaching!!!!!  j y suis retourn\xe9e sous un soleil de plomb (36\xb0) et mes recherches sont rest\xe9es infructueuses.La r\xe9ussite vient avec la pers\xe9v\xe9rance ....ca y est je l ai trouv\xe9e  en ce dimanche nuageux!!! Merci',
 u'Tres bonne cache: j ai mis un peu de temps!!!Pas facile avec la circulation de rester discr\xe8te .....Pr\xe9voir un stylo pour signer ',
 u'\U0001f600par cette belle journ\xe9e me voici parti me promener sur l \xeele .Bel endroit et bonne cache merci',
 u"Trouv\xe9e les 2 caches tr\xe8s rapidement :la 1 \xe8re n ayant pas de logbook j ai continu\xe9 mes recherches et enfin....mplc l endroit est tr\xe8s joli malgr\xe9 la route \xe0 proximit\xe9. N oubliez pas le stylo il n'y en a pas ",
 u'Sans trop de difficult\xe9s, tr\xe8s beau site \xe0 visiter.merci.',
 u'Tr\xe

In [249]:
logsSpeciaux={}

In [250]:
for log in listeLogs:
    if log[1] in logsSpeciaux:
        log[0]=logsSpeciaux[log[1]]
listeLogs.sort(key=lambda x: x[0])


In [251]:
logChars=set()
missingGrins=set()
for log in listeLogs:
    m=re.search(ur"(.*)\s*by (.*[aA]dopted\s+[bB]y.*)",log[1])
    if m:
        seps=[m.group(1),m.group(2)]
    else:
        seps=log[1].rsplit(u"by",1)
    log[1]=u"\\href{%s}{%s} — \\href{%s}{%d}\\\\{%s %s}"%(log[4],seps[0].strip(),log[5],log[0],log[3],seps[1].strip())
    for char in log[2]:
        logChars.add(char)
    m=re.findall(ur"(:[a-z]+?:)",log[2])
    if m:
        for element in m:
            missingGrins.add('u"%s"'%element)
print ", ".join(missingGrins)

In [252]:
import unicodedata,glob

pdfFiles=glob.glob("test/emoji_images/hires/*.pdf")
glyphs=set()
for pdf in pdfFiles:
    glyphNumber=pdf.rsplit("/",1)[1].rsplit(".",1)[0].lower()
    glyphs.add(glyphNumber)
    if "-" in glyphNumber:
        glyphs.add(glyphNumber.split("-")[0])

glyphs

{'1f53b',
 '1f53c',
 '1f53a',
 '1f53d',
 '1f645-1f3fc-2642',
 '1f645-1f3fb-2642',
 '34-20e3',
 '1f986',
 '1f4c3',
 '1f4c2',
 '1f4c1',
 '1f4c0',
 '1f4c7',
 '1f4c6',
 '1f4c5',
 '1f4c4',
 '1f4c9',
 '1f4c8',
 '1f356',
 '1f357',
 '1f354',
 '1f355',
 '1f352',
 '1f353',
 '1f350',
 '1f351',
 '1f9d2-1f3fb',
 '1f1ea',
 '1f1eb',
 '1f1ec',
 '1f455',
 '1f1ee',
 '1f457',
 '1f456',
 '1f925',
 '1f6ab',
 '1f927',
 '1f926',
 '1f921',
 '1f922',
 '1f36e',
 '1f361',
 '231b',
 '231a',
 '1f4cc',
 '1f4cb',
 '1f4ca',
 '2622',
 '1f4cf',
 '1f4ce',
 '1f4cd',
 '1f404',
 '1f532',
 '1f533',
 '1f530',
 '1f3cc-1f3fc',
 '1f536',
 '1f537',
 '1f534',
 '1f535',
 '1f538',
 '1f539',
 '1f30f',
 '2694-fe0f',
 '1f468-1f3fb-1f373',
 '1f92a',
 '23',
 '1f92b',
 '1f373',
 '2122',
 '1f35f',
 '1f1e9',
 '1f35d',
 '1f35e',
 '1f35b',
 '1f35c',
 '2b06',
 '1f35a',
 '1f45a',
 '1f45c',
 '1f45b',
 '1f45e',
 '1f45d',
 '1f1e6',
 '1f1e7',
 '1f3c0',
 '1f3c1',
 '1f3c2',
 '1f3c3',
 '1f3c4',
 '1f3c5',
 '1f3c6',
 '1f3c7',
 '1f575-1f3fb-2642',
 '1f3

In [253]:
missing=[]
for char in logChars:
    if not '%04x'%ord(char) in glyphs:
        try:
            if unicodedata.name(char) and ord(char)>255:
                missing.append(char)
        except:
            continue

print len(missing)
print "".join(sorted(missing))
for char in sorted(missing):
    print char,'%04x'%ord(char),unicodedata.name(char)

7
œ‍’…♀️￼
œ 0153 LATIN SMALL LIGATURE OE
‍ 200d ZERO WIDTH JOINER
’ 2019 RIGHT SINGLE QUOTATION MARK
… 2026 HORIZONTAL ELLIPSIS
♀ 2640 FEMALE SIGN
️ fe0f VARIATION SELECTOR-16
￼ fffc OBJECT REPLACEMENT CHARACTER


In [254]:
n=2000
print listeLogs[n][1]
print listeLogs[n][2]

\href{http://coord.info/GC6MCYZ}{Ou est le piège ? [AE2016]} — \href{http://coord.info/GC6MCYZ#795055253}{2001}\\{2018/08/16 Les GéotrouveTout44, Traditional Cache (3.5/2)}
Arrivés près du PZ nous tournons ,virons et finissons par mettre la main sur la cache sans camouflage. Le log book est détrempé: j’en  met un sec dans un petit sachet. Merci pour la cache.


In [255]:
print listeLogs[n][1]
print listeLogs[n][2]

\href{http://coord.info/GC6MCYZ}{Ou est le piège ? [AE2016]} — \href{http://coord.info/GC6MCYZ#795055253}{2001}\\{2018/08/16 Les GéotrouveTout44, Traditional Cache (3.5/2)}
Arrivés près du PZ nous tournons ,virons et finissons par mettre la main sur la cache sans camouflage. Le log book est détrempé: j’en  met un sec dans un petit sachet. Merci pour la cache.


## Filtrages

In [256]:
smileys={u"[:I]":u"😳",u"[8D]":u"😎",u"[:P]":u"😋",u"[;)]":u"😉",u"[8)]":u"🤭",
         u"[:)]":u"😀",u"[:(]":u"🤢",u"[xx(]":u"😵",u"[|)]":u"😴",u"[:D]":u"😁",
         u"[^]":u"🤩",u"[}:)]":u"😈",u"[:O]":u"😲",u"[:o)]":u"🤡",u"[B)]":u"🤕",
         u"[:8]":u"🎱",u"[8]":u"🎱",u"[:X]":u"😘",u"[?]":u"❓",u"[V]":u"😡",u"[:(!]":u"😡"}
smileysMatch="|".join([re.escape(k) for k in smileys.keys()])

def continuRe(listeLogs):
    fini=True
    for log in listeLogs:
        log[2]=log[2].replace(u"\\#",u"\\Number{}")
        log[2]=log[2].replace(u"[br]",u"\n")
        log[2]=log[2].replace(u"[b]*[/b]",u"[b]\\textbullet[/b]").replace(u"[/list=1]","[/list]")
        m=re.search(ur"(This entry was edited by .*on.*at\s+\d?\d:\d\d:\d\d\s*((AM|PM)?\s*UTC)?.)",log[2])
        if m:
            if debug: print "replacing edited by"
            log[2]=log[2].replace(m.group(1),"")
            fini=False
        m=re.findall(ur"(\[font=(.*?)\])",log[2])
        if m:
            if debug: print "replacing all [font]"
            for element in m:
                log[2]=log[2].replace(element[0],"")
            fini=False
        m=re.findall(ur"(\[url=(.*?)\](.*?)\[/url\])",log[2],re.DOTALL)
        if m:
            if debug: print "replacing all [url]"
            for element in m:
                if "\n" in element[2]:
                    elementHead,elementTail=element[2].split("\n",1)
                else:
                    elementHead=element[2]
                    elementTail=""
                log[2]=log[2].replace(element[0],u"\\href{%s}{%s}%s"%(element[1],elementHead,elementTail))
            fini=False
        m=re.findall(ur"(\[(blue|yellow|red|pink|orange|green|white|navy|maroon|gold|purple|teal|black)\]\s*(.*?)\s*\[/\2\](\s*))",log[2],re.DOTALL)
        if m:
            if debug: print "replacing all color"
            for element in m:
                log[2]=log[2].replace(element[0],u"\\color{%s}%s\\color{black}{%s}"%(element[1],element[2].strip(),element[3]))
            fini=False
        m=re.findall(ur"(\[size=(\d)\]\s*(.*?)\s*\[/size\])",log[2],re.DOTALL)
        if m:
            if debug: print "replacing all size"
            for element in m:
                size=element[1]
                if size=="4":
                    log[2]=log[2].replace(element[0],u"{\\LARGE{}%s}"%element[2])
                elif size=="3":
                    log[2]=log[2].replace(element[0],u"{\\Large{}%s}"%element[2])
                elif size=="2":
                    log[2]=log[2].replace(element[0],u"{\\large{}%s}"%element[2])
                else:
                    log[2]=log[2].replace(element[0],element[2])
            fini=False
        patron = re.compile(ur"(\[[lL][iI][sS][tT](=1)?\](((?!\[list).)*?)\[\/[lL][iI][sS][tT]\])",re.DOTALL)
        m=patron.findall(log[2])
        if m:
            print "replacing all [list]",log[0]
            for element in m:
                if log[0]==1531:
                    print element
                elementDeux=element[2]
                if element[0].startswith(u"[list=1]"):
                    log[2]=log[2].replace(element[0],u"\\begin{enumerate}\n%s\\end{enumerate}\n"%element[2])
                else:
                    log[2]=log[2].replace(element[0],u"\\begin{itemize}\n%s\\end{itemize}\n"%element[2])
                patronB=re.compile(ur"(\[\*\](.*?)\[/\*\])",re.DOTALL)
                b=patronB.findall(elementDeux)
                if b:
                    print "replacing all [*]"
                    for elb in b:
                        if log[0]==1531:
                            print "element",elementDeux
                            print "bullets",elb
                            print "log",log[2]
                        elementDeux=elementDeux.replace(elb[0],u"\\item %s\n"%elb[1])
                    log[2]=log[2].replace(element[2],elementDeux)
                else:
                    log[2]=log[2].replace(element[2],u"\\item %s\n"%element[2])
                if log[0]==1531:
                    print log[2]
            fini=False
        if log[0]==1531:
            print log[2]
        patron = re.compile(ur"\n(([\t ]*[\*>][ \t]+([^\n]+)\n)+)",re.DOTALL)
        m=patron.findall(log[2])
        if m:
            print "replacing all ^* $ lists",log[0]
            for element in m:
                b=re.findall(ur"^[ \t]*[\*>][ \t]+(.*)$",element[0],re.MULTILINE)
                if b:
                    itemize=u"\\begin{itemize}\n"
                    for bullet in b:
                        itemize=itemize+u"\\item %s\n"%bullet
                    itemize=itemize+u"\\end{itemize}\n"        
                    log[2]=log[2].replace(element[0],itemize)
            fini=False
        m=re.findall(ur"(\[s\](.*?)\[/s\])",log[2])
        if m:
            if debug: print "replacing all [strikethrough]"
            for element in m:
                log[2]=log[2].replace(element[0],u"\\sout{%s}"%element[1])
                fini=False
        m=re.findall(ur"(\[b\](.*?)\[/b\])",log[2])
        if m:
            if debug: print "replacing all [bold]"
            for element in m:
                log[2]=log[2].replace(element[0],u"{\\bfseries{}%s}"%element[1])
                fini=False
        m=re.search(ur"(\*\*(.*)\*\*)",log[2])
        if m:
            if debug: print "replacing **bold**"
            log[2]=log[2].replace(m.group(1),u"{\\bfseries{}%s}"%(m.group(2)))
            fini=False
        m=re.search(ur"(\[u\]([^[]*)\[/u\])",log[2],re.DOTALL)
        if m:
            if debug: print "replacing [underline]"
            log[2]=log[2].replace(m.group(1),u"{\\uline{%s}}"%(m.group(2)))
            fini=False
        m=re.search(ur"(\[i\]([^[]*)\[/ ?i\])",log[2],re.DOTALL)
        if m:
            if debug: print "replacing [italics]"
            log[2]=log[2].replace(m.group(1),u"{\\itshape{}%s}"%(m.group(2)))
            fini=False
        m=re.search(ur"(\*(.*?)\*)",log[2],re.MULTILINE)
        if m:
            if debug: print "replacing *italics*"
            log[2]=log[2].replace(m.group(1),u"{\\itshape{}%s}"%(m.group(2)))
            fini=False
        m=re.search(ur"\n(\*(.*?)\*.?)\n",log[2],re.DOTALL)
        if m:
            if debug: print "replacing *italics*"
            log[2]=log[2].replace(m.group(1),u"\n{\\itshape{}%s}\n"%(m.group(2)))
            fini=False
        m=re.search(ur"(%s)"%smileysMatch,log[2])
        if m:
            if debug: print "replacing smileys"
            if debug: print m.group(1)
            log[2]=log[2].replace(m.group(1),smileys["%s"%m.group(1)])
            fini=False
        patronM=re.compile(ur"(\[center\](.*?)\[/center\])",re.DOTALL)
        m=patronM.findall(log[2])
        if m:
            if debug: print "replacing [center]"
            for element in m:
                log[2]=log[2].replace(element[0],u"{\\begin{center}%s\\end{center}}"%element[1])
            fini=False
        patronM=re.compile(ur"^(###\s*(.*?)\s*###\s*)$",re.MULTILINE)
        m=patronM.findall(log[2])
        if m:
            if debug: print "replacing ### ###"
            for element in m:
                log[2]=log[2].replace(element[0],u"{\\centering\large\\color{green}%s\\color{black}\\par}"%element[1])
            fini=False
        patronM=re.compile(ur"^(##\s*(.*?)\s*##\s*)$",re.MULTILINE)
        m=patronM.findall(log[2])
        if m:
            if debug: print "replacing ## ##"
            for element in m:
                log[2]=log[2].replace(element[0],u"{\Large\\centering{}%s}"%element[1])
            fini=False
        patronM=re.compile(ur"^(#\s*(.*?)\s*#\s*)$",re.MULTILINE)
        m=patronM.findall(log[2])
        if m:
            if debug: print "replacing # #"
            for element in m:
                log[2]=log[2].replace(element[0],u"{\LARGE\\centering{}%s}"%element[1])
            fini=False
        result=fini
    return (listeLogs, result)

In [257]:
fini=False
while not fini:
    (listeLogs,fini)=continuRe(listeLogs)

Après le circuit de la bataille de Castillon, la pause déjeuner est très appréciée. Nous optons pour un petit circuit car la chaleur est étouffante et nous commençons à ressentir la fatigue. Direction Saint-Émilion...La balade est très agréable malgré la chaleur: des rencontres sympas, de gros fou rire et de très belles réalisations. Que du bonheur !!!!! . Nous loguons sous le pseudo Team 4 (Dune33 du 40,Fabilab du 44 et nous du 64).Merci pour la cache.


In [258]:
def continuReplace(listeLogs):
    fini=True
    for log in listeLogs:
        m=re.search(ur"(\[([^[]*)\]\(([^(]*)\))",log[2])
        if m:
            print "replacing [text](url)"
            log[2]=log[2].replace(m.group(1),u"\\href{%s}{%s}"%(m.group(3),m.group(2)))
            fini=False
        m=re.search(ur"(#\s?(\d+\.?))",log[2])
        if m:
            print "replacing #num."
            log[2]=log[2].replace(m.group(1),u"\\Number{}%s"%m.group(2))
            fini=False
        m=re.findall(ur'("(.+)")',log[2])
        if m:
            print "replacing double quotes."
            for element in m:
                log[2]=log[2].replace(element[0],u"\\Quoted{%s}"%element[1].strip())
            fini=False
        m=re.findall(ur'("\s*(.+)\s*")',log[1])
        if m:
            print "replacing double quotes."
            for element in m:
                log[1]=log[1].replace(element[0],u"\\Quoted{%s}"%element[1].strip())
            fini=False
        m=re.search(ur"(#(%d|{\\bfseries|\s*\\color|MMCXI))"%log[0],log[2])
        if m:
            print "replacing #num."
            log[2]=log[2].replace(m.group(1),u"%s"%m.group(2))
            fini=False
    result=fini
    return (listeLogs, result)

In [259]:
fini=False
while not fini:
    (listeLogs,fini)=continuReplace(listeLogs)

replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing #num.
replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing double quotes.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing double quotes.
replacing #num.
replacing #num.
replacing double quotes.
replacing #num.
replacing d

In [260]:
import struct

def unichar(i):
    '''https://stackoverflow.com/questions/7105874/valueerror-unichr-arg-not-in-range0x10000-narrow-python-build'''
    try:
        return unichr(i)
    except ValueError:
        return struct.pack('i', i).decode('utf-32')
    
def getGlyph(hexCode):
    uniCode=int(hexCode,16)
    return unichar(uniCode)

In [261]:
longGlyphs=[(u"🇫🇷",u"1F1EB-1F1F7"),
            (u"🇪🇸",u"1F1EA-1F1F8"),
            (u"🇲🇾",u"1F1F2-1F1FE"),
            (u"🇸🇬",u"1F1F8-1F1EC"),
            (u"🇬🇧",u"1F1EC-1F1E7"),
            (u"🇺🇸",u"1F1FA-1F1F8"),
            (u"🇵🇹",u"1F1F5-1F1F9"),
            (u"🇧🇪",u"1F1E7-1F1EA"),
            (u"🇩🇪",u"1F1E9-1F1EA"),
            (u"🇳🇱",u"1F1F3-1F1F1"),
            (u"🇸🇪",u"1F1F8-1F1EA"),
            (u"🇩🇰",u"1F1E9-1F1F0"),
            (u"🇨🇭",u"1F1E8-1F1ED"),
            (u"🇦🇺",u"1F1E6-1F1FA"),
            (u"🇳🇿",u"1F1F3-1F1FF"),
            (u"🇨🇦",u"1F1E8-1F1E6"),
            (u"🇮🇳",u"1F1EE-1F1F3"),
            (u"🇳🇵",u"1F1F3-1F1F5"),
            (u"🇻🇦",u"1F1FB-1F1E6"),
            (u"🇦🇩",u"1F1E6-1F1E9"),
            (u"🇬🇳",u"1F1EC-1F1F3"),
            (u"🇮🇩",u"1F1EE-1F1E9"),
            (u"🇧🇷",u"1F1E7-1F1F7"),
            (u"🇵🇪",u"1F1F5-1F1EA"),
            (u"🇦🇷",u"1F1E6-1F1F7"),
            (u"🇻🇪",u"1F1FB-1F1EA"),
            (u"🇹🇷",u"1F1F9-1F1F7"),
            (u"🇪🇬",u"1F1EA-1F1EC"),
            (u"🇷🇺",u"1F1F7-1F1FA"),
            (u"🇨🇳",u"1F1E8-1F1F3"),
            (u"🇭🇺",u"1F1ED-1F1FA"),
            (u"🇮🇱",u"1F1EE-1F1F1"),
            (u"🇮🇸",u"1F1EE-1F1F8")
           ]

In [262]:
def getCodePoint(char):
    if len(char) ==1 :
        result= ord(char)
    elif len(char) ==2 :
        result= 0x10000 + (ord(char[0]) - 0xD800) * 0x400 + (ord(char[1]) - 0xDC00)
    else:
        print "length not in [1,2]"
        result=""
    return ("%8x"%result).strip().upper()

In [263]:
coloredGlyphs=(u"☁︎,⚱️,☮️,✝️,♒️,🕉,☂️,⛄️,☄️,☀️,☘️,☠️,☢️,☣️,✍️"
               u",▪️,◾️,◼️,⬛️,▫️,◽️,◻️,⬜️,❣️,"
               u"☔️,🕹,🖍,🌧,🏵,🖥,🖼,🏖,🏟,🕊,"
               u"🍽,🗒,🌬,🌤,⛅️,🌥,🌨,🕸,🏘,🛰,🛡,⏱,⚙️,🎖,⏹️,☎️,🛎,⚖️,⚔️"
              )
for glyph in coloredGlyphs.split(","):
    glyphPoint=getCodePoint(glyph)
    if glyphPoint.startswith("-"):
        glyphCode=[]
        for char in glyph:
#            print getCodePoint(char)
            glyphCode.append(getCodePoint(char))
        print glyph,"-".join(glyphCode)
        longGlyphs.append((glyph,"-".join(glyphCode)))
    else:
        print glyph,"%s-FE0F"%glyphPoint
        longGlyphs.append((glyph,"%s-FE0F"%glyphPoint))
    
#    

☁︎ 2601-FE0E
⚱️ 26B1-FE0F
☮️ 262E-FE0F
✝️ 271D-FE0F
♒️ 2652-FE0F
🕉 1F549-FE0F
☂️ 2602-FE0F
⛄️ 26C4-FE0F
☄️ 2604-FE0F
☀️ 2600-FE0F
☘️ 2618-FE0F
☠️ 2620-FE0F
☢️ 2622-FE0F
☣️ 2623-FE0F
✍️ 270D-FE0F
▪️ 25AA-FE0F
◾️ 25FE-FE0F
◼️ 25FC-FE0F
⬛️ 2B1B-FE0F
▫️ 25AB-FE0F
◽️ 25FD-FE0F
◻️ 25FB-FE0F
⬜️ 2B1C-FE0F
❣️ 2763-FE0F
☔️ 2614-FE0F
🕹 1F579-FE0F
🖍 1F58D-FE0F
🌧 1F327-FE0F
🏵 1F3F5-FE0F
🖥 1F5A5-FE0F
🖼 1F5BC-FE0F
🏖 1F3D6-FE0F
🏟 1F3DF-FE0F
🕊 1F54A-FE0F
🍽 1F37D-FE0F
🗒 1F5D2-FE0F
🌬 1F32C-FE0F
🌤 1F324-FE0F
⛅️ 26C5-FE0F
🌥 1F325-FE0F
🌨 1F328-FE0F
🕸 1F578-FE0F
🏘 1F3D8-FE0F
🛰 1F6F0-FE0F
🛡 1F6E1-FE0F
⏱ 23F1-FE0F
⚙️ 2699-FE0F
🎖 1F396-FE0F
⏹️ 23F9-FE0F
☎️ 260E-FE0F
🛎 1F6CE-FE0F
⚖️ 2696-FE0F
⚔️ 2694-FE0F


In [264]:
sexVariation={u"\u2642":u"-2642",u"\u2640":u"-2640"}
skinColor={u"\U0001F3FB":"-1F3FB",u"\U0001F3FC":u"-1F3FC",u"\U0001F3FD":"-1F3FD",
       u"\U0001F3FE":"-1F3FE",u"\U0001F3FF":u"-1F3FF",u"\uFE0F":""}
skinColors=[u"\U0001F3FB",u"\U0001F3FC",u"\U0001F3FD",u"\U0001F3FE",u"\U0001F3FF",u"\uFE0F"]


In [265]:
def addSkinTone(baseCode,modifier=u"\u200D",suffix=""):
    baseGlyph=getGlyph(baseCode)
    if suffix!="":
        suffixGlyph=getGlyph(suffix)
        suffix=u"-"+suffix
        print "suffix",suffixGlyph
    else:
        suffixGlyph=""
    for color in skinColors:
        if color!=u"\uFE0F":
            longGlyphs.append((baseGlyph+color+modifier+suffixGlyph,baseCode+skinColor[color]+suffix))
        else:
            longGlyphs.append((baseGlyph+modifier+suffixGlyph,baseCode+suffix))
    longGlyphs.append((baseGlyph+suffixGlyph,baseCode+suffix))
    return


def addSkinToneSex(baseCode,suffix=""):
    baseGlyph=getGlyph(baseCode)
    if suffix!="":
        suffixGlyph=getGlyph(suffix)
        suffix=u"-"+suffix
    else:
        suffixGlyph=""
    for color in skinColors:
        for sex in sexVariation:
            if color!=u"\uFE0F":
                temp=(baseGlyph+color+u"\u200D"+sex+u"\uFE0F"+suffixGlyph,baseCode+skinColor[color]+sexVariation[sex]+suffix)
                longGlyphs.append(temp)
            else:
                temp=(baseGlyph+u"\u200D"+sex+u"\uFE0F"+suffixGlyph,baseCode+sexVariation[sex]+suffix)
                longGlyphs.append(temp)
        if color!=u"\uFE0F":
            temp=(baseGlyph+color+suffixGlyph,baseCode+skinColor[color]+suffix)
            longGlyphs.append(temp)
        else:
            temp=(baseGlyph+suffixGlyph,baseCode+suffix)
            longGlyphs.append(temp)    
    return

In [266]:
for suffix in u"🌾,🏫,🍳,💻,💼,🚒,\u2695,\u2696,🔬,\u2708".split(","):
    codePointSuffix=getCodePoint(suffix)
    for char in u"👨,👩".split(","):
        codePoint=getCodePoint(char)
        print suffix,char,codePoint
        addSkinTone(codePoint,suffix=codePointSuffix)

🌾 👨 1F468
suffix 🌾
🌾 👩 1F469
suffix 🌾
🏫 👨 1F468
suffix 🏫
🏫 👩 1F469
suffix 🏫
🍳 👨 1F468
suffix 🍳
🍳 👩 1F469
suffix 🍳
💻 👨 1F468
suffix 💻
💻 👩 1F469
suffix 💻
💼 👨 1F468
suffix 💼
💼 👩 1F469
suffix 💼
🚒 👨 1F468
suffix 🚒
🚒 👩 1F469
suffix 🚒
⚕ 👨 1F468
suffix ⚕
⚕ 👩 1F469
suffix ⚕
⚖ 👨 1F468
suffix ⚖
⚖ 👩 1F469
suffix ⚖
🔬 👨 1F468
suffix 🔬
🔬 👩 1F469
suffix 🔬
✈ 👨 1F468
suffix ✈
✈ 👩 1F469
suffix ✈


In [267]:
for char in u"✍,🤴,👶,👧,🧒,👩,🙎,👵,👰,👸,🎅,👴,🤳,🖐,✌,🧙,💃,🤚".split(","):
    codePoint=getCodePoint(char)
    print char,codePoint
    addSkinTone(codePoint,modifier="")

✍ 270D
🤴 1F934
👶 1F476
👧 1F467
🧒 1F9D2
👩 1F469
🙎 1F64E
👵 1F475
👰 1F470
👸 1F478
🎅 1F385
👴 1F474
🤳 1F933
🖐 1F590
✌ 270C
🧙 1F9D9
💃 1F483
🤚 1F91A


In [268]:
for char in u"⛹,🤸,🚶,🏃,👱,🚴,👳,\U0001F575,💂,💇,🙅,🤽,👮,👴,💁,🤹,🙋,🤷,🏊,\U0001F3CC,🙍,👦,🤦".split(","):
    codePoint=getCodePoint(char)
    print char,codePoint
    addSkinToneSex(codePoint)

⛹ 26F9
🤸 1F938
🚶 1F6B6
🏃 1F3C3
👱 1F471
🚴 1F6B4
👳 1F473
🕵 1F575
💂 1F482
💇 1F487
🙅 1F645
🤽 1F93D
👮 1F46E
👴 1F474
💁 1F481
🤹 1F939
🙋 1F64B
🤷 1F937
🏊 1F3CA
🏌 1F3CC
🙍 1F64D
👦 1F466
🤦 1F926


In [269]:
getCodePoint(u"🖼")

'1F5BC'

In [270]:
for element in longGlyphs:
    print element[0],element[1]


🇫🇷 1F1EB-1F1F7
🇪🇸 1F1EA-1F1F8
🇲🇾 1F1F2-1F1FE
🇸🇬 1F1F8-1F1EC
🇬🇧 1F1EC-1F1E7
🇺🇸 1F1FA-1F1F8
🇵🇹 1F1F5-1F1F9
🇧🇪 1F1E7-1F1EA
🇩🇪 1F1E9-1F1EA
🇳🇱 1F1F3-1F1F1
🇸🇪 1F1F8-1F1EA
🇩🇰 1F1E9-1F1F0
🇨🇭 1F1E8-1F1ED
🇦🇺 1F1E6-1F1FA
🇳🇿 1F1F3-1F1FF
🇨🇦 1F1E8-1F1E6
🇮🇳 1F1EE-1F1F3
🇳🇵 1F1F3-1F1F5
🇻🇦 1F1FB-1F1E6
🇦🇩 1F1E6-1F1E9
🇬🇳 1F1EC-1F1F3
🇮🇩 1F1EE-1F1E9
🇧🇷 1F1E7-1F1F7
🇵🇪 1F1F5-1F1EA
🇦🇷 1F1E6-1F1F7
🇻🇪 1F1FB-1F1EA
🇹🇷 1F1F9-1F1F7
🇪🇬 1F1EA-1F1EC
🇷🇺 1F1F7-1F1FA
🇨🇳 1F1E8-1F1F3
🇭🇺 1F1ED-1F1FA
🇮🇱 1F1EE-1F1F1
🇮🇸 1F1EE-1F1F8
☁︎ 2601-FE0E
⚱️ 26B1-FE0F
☮️ 262E-FE0F
✝️ 271D-FE0F
♒️ 2652-FE0F
🕉 1F549-FE0F
☂️ 2602-FE0F
⛄️ 26C4-FE0F
☄️ 2604-FE0F
☀️ 2600-FE0F
☘️ 2618-FE0F
☠️ 2620-FE0F
☢️ 2622-FE0F
☣️ 2623-FE0F
✍️ 270D-FE0F
▪️ 25AA-FE0F
◾️ 25FE-FE0F
◼️ 25FC-FE0F
⬛️ 2B1B-FE0F
▫️ 25AB-FE0F
◽️ 25FD-FE0F
◻️ 25FB-FE0F
⬜️ 2B1C-FE0F
❣️ 2763-FE0F
☔️ 2614-FE0F
🕹 1F579-FE0F
🖍 1F58D-FE0F
🌧 1F327-FE0F
🏵 1F3F5-FE0F
🖥 1F5A5-FE0F
🖼 1F5BC-FE0F
🏖 1F3D6-FE0F
🏟 1F3DF-FE0F
🕊 1F54A-FE0F
🍽 1F37D-FE0F
🗒 1F5D2-FE0F
🌬 1F32C-FE0F
🌤 1F324-FE0F
⛅️ 26C5-FE0

🕵‍♀️ 1F575-2640
🕵‍♂️ 1F575-2642
🕵 1F575
💂🏻‍♀️ 1F482-1F3FB-2640
💂🏻‍♂️ 1F482-1F3FB-2642
💂🏻 1F482-1F3FB
💂🏼‍♀️ 1F482-1F3FC-2640
💂🏼‍♂️ 1F482-1F3FC-2642
💂🏼 1F482-1F3FC
💂🏽‍♀️ 1F482-1F3FD-2640
💂🏽‍♂️ 1F482-1F3FD-2642
💂🏽 1F482-1F3FD
💂🏾‍♀️ 1F482-1F3FE-2640
💂🏾‍♂️ 1F482-1F3FE-2642
💂🏾 1F482-1F3FE
💂🏿‍♀️ 1F482-1F3FF-2640
💂🏿‍♂️ 1F482-1F3FF-2642
💂🏿 1F482-1F3FF
💂‍♀️ 1F482-2640
💂‍♂️ 1F482-2642
💂 1F482
💇🏻‍♀️ 1F487-1F3FB-2640
💇🏻‍♂️ 1F487-1F3FB-2642
💇🏻 1F487-1F3FB
💇🏼‍♀️ 1F487-1F3FC-2640
💇🏼‍♂️ 1F487-1F3FC-2642
💇🏼 1F487-1F3FC
💇🏽‍♀️ 1F487-1F3FD-2640
💇🏽‍♂️ 1F487-1F3FD-2642
💇🏽 1F487-1F3FD
💇🏾‍♀️ 1F487-1F3FE-2640
💇🏾‍♂️ 1F487-1F3FE-2642
💇🏾 1F487-1F3FE
💇🏿‍♀️ 1F487-1F3FF-2640
💇🏿‍♂️ 1F487-1F3FF-2642
💇🏿 1F487-1F3FF
💇‍♀️ 1F487-2640
💇‍♂️ 1F487-2642
💇 1F487
🙅🏻‍♀️ 1F645-1F3FB-2640
🙅🏻‍♂️ 1F645-1F3FB-2642
🙅🏻 1F645-1F3FB
🙅🏼‍♀️ 1F645-1F3FC-2640
🙅🏼‍♂️ 1F645-1F3FC-2642
🙅🏼 1F645-1F3FC
🙅🏽‍♀️ 1F645-1F3FD-2640
🙅🏽‍♂️ 1F645-1F3FD-2642
🙅🏽 1F645-1F3FD
🙅🏾‍♀️ 1F645-1F3FE-2640
🙅🏾‍♂️ 1F645-1F3FE-2642
🙅🏾 1F645-1F3FE
🙅🏿‍♀️ 1F645-1F3FF-2640
🙅🏿‍

In [271]:
grinGlyphs={
            u":grin:":u"😃", u":sad:":u"😒", 
            u":yikes:":u"🤮", u":tongue:":u"😛", 
            u":smile:":u"🙂", u":shocked:":u"🤭", 
            u":surprise:":u"😮", u":tired:":u"😴", 
            u":cute:":u"😳", u":angry:":u"😡", u":back:":u"🌕"
}

In [272]:
setGlyphs=[]
for log in listeLogs:
    log[1]=log[1].replace(u"&nbsp;",ur" ").replace("#","\\Number{}").replace("_","\\Underscore{}")
    log[1]=log[1].replace("&","\\And{}").replace("^","\\textasciicircum")
    log[1]=log[1].replace(u"«","\\og{}").replace(u"»","\\fg{}")
    log[2]=log[2].replace(u"«","\\og{}").replace(u"»","\\fg{}")
    log[2]=log[2].replace(u"\n==EN= ",u"\n----\n🇬🇧\n").replace(u"\n==EN== ",u"\n----\n🇬🇧\n")
    log[2]=log[2].replace(u"\n\n\n",u"\n\\smallskip\n\n")
    log[2]=log[2].replace(u"\n",u"\n\n")
    log[2]=log[2].replace(u"\n----\n",u"\n---\n")
    log[2]=log[2].replace(u"\n---\n",u"\n\\rule{8cm}{0.4pt}\n\\smallskip\n\n")
    log[2]=log[2].replace(u"&nbsp;",ur" ").replace("&","\\And{}").replace("_","\\Underscore{}")
    log[2]=log[2].replace("^","\\textasciicircum").replace("$","\\textdollar{}").replace("%","\\Percent{}")
    log[2]=log[2].replace(u"सुजीव",u"{\\sanskritfont सुजीव}")
    log[2]=log[2].replace(u"वेरोक्स",u"{\\sanskritfont वेरोक्स}")
    log[2]=log[2].replace(u"ૐ",u"{\\sanskritfont ॐ}")
    log[2]=log[2].replace(u"Αρκούδες",u"{\\greekfont Αρκούδες}")
    log[2]=log[2].replace(u"εστιατόριο",u"{\\greekfont εστιατόριο}")
    log[2]=log[2].replace(u"μεζές",u"{\\greekfont μεζές}")
    log[2]=log[2].replace(u"μουσακάς",u"{\\greekfont μουσακάς}")
    log[2]=log[2].replace(u"θηλειά άρκτος",u"{\\greekfont θηλειά άρκτος}")
    log[2]=log[2].replace(u"2603#",u"2603")
    m=re.findall(ur"(\\href{(\\\w+?\{\}|[^{}]*)+})",log[2])
    if m:
        for element in m:
            href=element[0].replace(u"\\Underscore{}",u"\\_").replace(u"\\And{}",u"\\&").replace(u"\\Percent{}",u"\\%").replace(u"\\Number{}",u"\\#")
            log[2]=log[2].replace(element[0],href)
    for charGlyph in longGlyphs:
        char=charGlyph[0]
        glyph=charGlyph[1]
        if char in log[2]:
            if not char in setGlyphs:
                if not glyph.lower() in glyphs:
                    print char,glyph
                setGlyphs.append(char)
            log[2]=log[2].replace(char,u"\\includegraphics{./emoji_images/hires/%s.pdf}"%glyph)
    for grin in grinGlyphs:
        log[2]=log[2].replace(grin,grinGlyphs[grin])

In [273]:
chapters={1:u"Débuts"
         }
milestones={
         }

In [274]:
#indexPseudos=u"Calimero33 Crispol40 DeBDX Domino50 DorisBear Dune33 Fdcdm Fleura64 Gaingain Gilles64 IsaAsia Loango MadanneGéo Mizaga MKL33210 MO13Luis Nanard470  Patricia64 Opmb40 Peyo64 Plume64 Puma RBTGP Sanglier6440 SenninhaN.R. Sod@'s Yvain64 Zam's"
#dictPseudos={k:[k.lower()] for k in indexPseudos.split(" ") if k!=""}

In [275]:
#indexStringObj=u"Aube Aurore Billy Bolly Cachou Camel Carlo Celion Coralie Dora Dragon Fab Festif Fête Fonce GyPSy Kadia Larruz Leonardo Lorea Lucie Ludo Marcia Michèle Minnie Nelly Pétale Pilar Stéphane Steve Thomas Tim Tima Tina Trouve Vanessa Viccar Victoria Victure Violette Virginie Willy"
#listObjets=[k for k in indexStringObj.split(" ") if k!=""]

In [276]:
listObjets=[
            ]

In [277]:
dictPseudos={}

In [278]:
def makeIndex(log):
    result=log
    for k in dictPseudos:
        for v in dictPseudos[k]:
            result=re.sub(ur"(?<!le/\?u=|esker | TFTC )("+v+"|"+v.capitalize()+"|"+v.upper()+")","\\g<1>\\index{%s}"%(k),result)
    for obj in listObjets:
        result=re.sub(ur"(^|\s)("+obj+ur")([\s.,])","\\g<1>\\g<2>\\index{\\g<2>}\\g<3>",result,re.DOTALL)
    return result
        

In [279]:
makeIndex(u"Ludo, Mark et Bolly ont rencontré Sod@'s avec TFTC Zam's... milesker Gaingain était en scoot, profile/?u=Peyo64 en voiture.")

u"Ludo, Mark et Bolly ont rencontr\xe9 Sod@'s avec TFTC Zam's... milesker Gaingain \xe9tait en scoot, profile/?u=Peyo64 en voiture."

In [280]:
lines=[]
for l in listeLogs:
    if l[0]>4000:
        pass
    else:
        if l[0] in chapters:
            lines.append(u"\\chapter{%s}"%chapters[l[0]])
        lines.append(u"\\cacheNumber{%s}"%l[0])
        lines.append(u"\\needspace{5\\baselineskip}")
        cacheName,cacheData=l[1].split("\\\\")
        if l[0] in milestones:
            lines.append(u"\\cacheName{%s\\mileStone{%s}}\\cacheData{%s}"%(cacheName,milestones[l[0]],cacheData))
        else:
            lines.append(u"\\cacheName{%s}\\cacheData{%s}"%(cacheName,cacheData))
        lines.append(u"\\begin{cacheText}")
        lines.append(makeIndex(l[2].strip()))
        lines.append(u"\\end{cacheText}\n\n")

In [281]:
print len(lines)

14215


In [282]:
with codecs.open("test/TextesLog-Fleura.tex","w",encoding='utf8') as output:
    output.writelines(lines[:5000])
    output.writelines(lines[5000:10000])
    output.writelines(lines[10000:15000])
    output.writelines(lines[15000:])

# FIN

In [73]:
test=ur'''\href{http://coord.info/GC3FW3X}{GR10#152} — \href{http://coord.info/GC3FW3X#352424828}{1531}\\{2013/09/22 Peyo64, Traditional Cache (2/1.5)}
#1531. Nouvelle expédition pour le segment mythique du GR10-04 avec les Robos (Mr Coach [8D] & Mme Défi [:I]) et une invitée surprise (Dame Nanas [8)]) pour compléter la moitié restante des caches autour d'Iparla. Cette semaine, la victure est de repos et c'est la Robocar qui assure le transport de l'équipe. Après une matinée laborieuse, l'équipe a bien profité de la pause déjeuner à l'ombre pour échafauder des plans pour l'après-midi...

Plusieurs options sont envisagées juste après manger :[list=1][*]continuer jusqu'à Iparla[/*][list][*]revenir par le même chemin[/*][*]envoyer le Coach déplacer la Robocar[/*][list][*]revenir par Izpegi[/*][*]continuer jusqu'à Bidarray[/*][/list][/list][*]rebrousser chemin[/*][list][*]revenir par le même chemin[/*][*]envoyer le Coach déplacer la Robocar à Izpegi[/*][/list][/list]Pour finir, la décision est remise à plus tard et on entame la descente jusqu'à la cache suivante. Pendant que Mme Défi et le double s'affairent avec le Dragon, Dame Nanas découvre sa première ampoule. MPLC, milesker

After lunch, our team wonders what's next. It's already a bit late and we're still somewhat far from Iparla. The signs say it's gonna take us quite some time to double back, so we hesitate. After a while we decide to push on for one or two caches more and see where we get from there... This first one is also Nanas' first discovery. Hail to the newbie...! TFTC

No Trade'''

In [74]:
print test
print

patron = re.compile(ur"(\[list(=1)?\](((?!\[list).)*?)\[\/list\])",re.DOTALL)
m=patron.findall(test)
if m:
    for element in m:
        print "element",element
        elementZero=element[2]
        if element[0].startswith(u"[list=1]"):
            test=test.replace(element[0],u"\\begin{enumerate}\n%s\\end{enumerate}\n"%element[2])
        else:
            test=test.replace(element[0],u"\\begin{itemize}\n%s\\end{itemize}\n"%element[2])
        patronB=re.compile(ur"(\[\*\](.*?)\[\/\*\])",re.DOTALL)
        b=patronB.findall(elementZero)
        if b:
            print "replacing all [*]",elementZero
            for elb in b:
                print "elb",elb
                elementZero=elementZero.replace(elb[0],u"\\item %s\n"%elb[1])
            print "Zero",elementZero
            print "Orig",element[2]
            test=test.replace(element[2],elementZero)
        else:
            test=test.replace(element[2],u"\\item %s\n"%element[2])
    fini=False
print test

\href{http://coord.info/GC3FW3X}{GR10#152} — \href{http://coord.info/GC3FW3X#352424828}{1531}\\{2013/09/22 Peyo64, Traditional Cache (2/1.5)}
#1531. Nouvelle expédition pour le segment mythique du GR10-04 avec les Robos (Mr Coach [8D] & Mme Défi [:I]) et une invitée surprise (Dame Nanas [8)]) pour compléter la moitié restante des caches autour d'Iparla. Cette semaine, la victure est de repos et c'est la Robocar qui assure le transport de l'équipe. Après une matinée laborieuse, l'équipe a bien profité de la pause déjeuner à l'ombre pour échafauder des plans pour l'après-midi...

Plusieurs options sont envisagées juste après manger :[list=1][*]continuer jusqu'à Iparla[/*][list][*]revenir par le même chemin[/*][*]envoyer le Coach déplacer la Robocar[/*][list][*]revenir par Izpegi[/*][*]continuer jusqu'à Bidarray[/*][/list][/list][*]rebrousser chemin[/*][list][*]revenir par le même chemin[/*][*]envoyer le Coach déplacer la Robocar à Izpegi[/*][/list][/list]Pour finir, la décision est remis

In [75]:
test=u"We did some bikepooling to get there. Thanks to (M)Obike 🚲 we got two mounts (same-same but different [;)]) close enough to our hotel 🤽‍♂️ to ride all the way to the swimming pool 🏊🏻 (closed but really attractive). The site was not too busy and even featured a bicycle parking lot. The local 🤽‍♂️ trainer 🤽‍♂️ was very helpful in pointing out that we could not use the pool but we might be allowed by the guard 👮🏽 visit the premises. We 🕵🏻 declined and went for our own kind of treasure."

In [76]:
longGlyphs

[(u'\U0001f1eb\U0001f1f7', u'1F1EB-1F1F7'),
 (u'\U0001f1ea\U0001f1f8', u'1F1EA-1F1F8'),
 (u'\U0001f1f2\U0001f1fe', u'1F1F2-1F1FE'),
 (u'\U0001f1f8\U0001f1ec', u'1F1F8-1F1EC'),
 (u'\U0001f1ec\U0001f1e7', u'1F1EC-1F1E7'),
 (u'\U0001f1fa\U0001f1f8', u'1F1FA-1F1F8'),
 (u'\U0001f1f5\U0001f1f9', u'1F1F5-1F1F9'),
 (u'\U0001f1e7\U0001f1ea', u'1F1E7-1F1EA'),
 (u'\U0001f1e9\U0001f1ea', u'1F1E9-1F1EA'),
 (u'\U0001f1f3\U0001f1f1', u'1F1F3-1F1F1'),
 (u'\U0001f1f8\U0001f1ea', u'1F1F8-1F1EA'),
 (u'\U0001f1e9\U0001f1f0', u'1F1E9-1F1F0'),
 (u'\U0001f1e8\U0001f1ed', u'1F1E8-1F1ED'),
 (u'\U0001f1e6\U0001f1fa', u'1F1E6-1F1FA'),
 (u'\U0001f1f3\U0001f1ff', u'1F1F3-1F1FF'),
 (u'\U0001f1e8\U0001f1e6', u'1F1E8-1F1E6'),
 (u'\U0001f1ee\U0001f1f3', u'1F1EE-1F1F3'),
 (u'\U0001f1f3\U0001f1f5', u'1F1F3-1F1F5'),
 (u'\U0001f1fb\U0001f1e6', u'1F1FB-1F1E6'),
 (u'\U0001f1e6\U0001f1e9', u'1F1E6-1F1E9'),
 (u'\U0001f1ec\U0001f1f3', u'1F1EC-1F1F3'),
 (u'\U0001f1ee\U0001f1e9', u'1F1EE-1F1E9'),
 (u'\U0001f1e7\U0001f1f7', u'1F1

In [77]:
pos=test.index(u'\U0001F93D')
print getCodePoint(test[pos:pos+2]),getCodePoint(test[pos+2]),getCodePoint(test[pos+3]),getCodePoint(test[pos+4]),getCodePoint(test[pos+5])
print getCodePoint(test[pos:pos+2]),getCodePoint(test[pos+2:pos+4]),getCodePoint(test[pos+4]),getCodePoint(test[pos+5])
print getCodePoint(test[pos]),getCodePoint(test[pos+1]),getCodePoint(test[pos+2]),getCodePoint(test[pos+3]),getCodePoint(test[pos+4])


1F93D 200D 2642 FE0F 20
1F93D -2DF81BE FE0F 20
D83E DD3D 200D 2642 FE0F


In [78]:
for charGlyph in longGlyphs:
    char=charGlyph[0]
    glyph=charGlyph[1]
    print [char],glyph
    if char in test:
        if not char in setGlyphs:
            if not glyph.lower() in glyphs:
                print char,glyph
            setGlyphs.append(char)
        test=test.replace(char,u"\\includegraphics{./emoji_images/hires/%s.pdf}"%glyph)
test

[u'\U0001f1eb\U0001f1f7'] 1F1EB-1F1F7
[u'\U0001f1ea\U0001f1f8'] 1F1EA-1F1F8
[u'\U0001f1f2\U0001f1fe'] 1F1F2-1F1FE
[u'\U0001f1f8\U0001f1ec'] 1F1F8-1F1EC
[u'\U0001f1ec\U0001f1e7'] 1F1EC-1F1E7
[u'\U0001f1fa\U0001f1f8'] 1F1FA-1F1F8
[u'\U0001f1f5\U0001f1f9'] 1F1F5-1F1F9
[u'\U0001f1e7\U0001f1ea'] 1F1E7-1F1EA
[u'\U0001f1e9\U0001f1ea'] 1F1E9-1F1EA
[u'\U0001f1f3\U0001f1f1'] 1F1F3-1F1F1
[u'\U0001f1f8\U0001f1ea'] 1F1F8-1F1EA
[u'\U0001f1e9\U0001f1f0'] 1F1E9-1F1F0
[u'\U0001f1e8\U0001f1ed'] 1F1E8-1F1ED
[u'\U0001f1e6\U0001f1fa'] 1F1E6-1F1FA
[u'\U0001f1f3\U0001f1ff'] 1F1F3-1F1FF
[u'\U0001f1e8\U0001f1e6'] 1F1E8-1F1E6
[u'\U0001f1ee\U0001f1f3'] 1F1EE-1F1F3
[u'\U0001f1f3\U0001f1f5'] 1F1F3-1F1F5
[u'\U0001f1fb\U0001f1e6'] 1F1FB-1F1E6
[u'\U0001f1e6\U0001f1e9'] 1F1E6-1F1E9
[u'\U0001f1ec\U0001f1f3'] 1F1EC-1F1F3
[u'\U0001f1ee\U0001f1e9'] 1F1EE-1F1E9
[u'\U0001f1e7\U0001f1f7'] 1F1E7-1F1F7
[u'\U0001f1f5\U0001f1ea'] 1F1F5-1F1EA
[u'\U0001f1e6\U0001f1f7'] 1F1E6-1F1F7
[u'\U0001f1fb\U0001f1ea'] 1F1FB-1F1EA
[u'\U0001f1f

u'We did some bikepooling to get there. Thanks to (M)Obike \U0001f6b2 we got two mounts (same-same but different [;)]) close enough to our hotel \\includegraphics{./emoji_images/hires/1F93D-2642.pdf} to ride all the way to the swimming pool \\includegraphics{./emoji_images/hires/1F3CA-1F3FB.pdf} (closed but really attractive). The site was not too busy and even featured a bicycle parking lot. The local \\includegraphics{./emoji_images/hires/1F93D-2642.pdf} trainer \\includegraphics{./emoji_images/hires/1F93D-2642.pdf} was very helpful in pointing out that we could not use the pool but we might be allowed by the guard \\includegraphics{./emoji_images/hires/1F46E-1F3FD.pdf} visit the premises. We \\includegraphics{./emoji_images/hires/1F575-1F3FB.pdf} declined and went for our own kind of treasure.'